In [1]:
import graphlab
import base64

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
openfood_sf = graphlab.SFrame('http://s3.amazonaws.com/dato-datasets/food/filtered_openfood_sf')
dedup = graphlab.nearest_neighbor_deduplication.create(openfood_sf, features=['product_name'],radius=0.25)
dedup_sf = dedup['entities'].groupby(key_columns="__entity", operations = {'row_number' : graphlab.aggregate.SELECT_ONE('row_number')})
openfood_sf = openfood_sf.add_row_number('row_number').filter_by(dedup_sf['row_number'], 'row_number')


[INFO] This non-commercial license of GraphLab Create is assigned to kennethcc2005@hotmail.com and will expire on February 10, 2017. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-63194 - Server binary: /Users/zoesh/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1459983091.log
[INFO] GraphLab Server Version: 1.8.1


PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/food/filtered_openfood_sf/dir_archive.ini to /var/tmp/graphlab-zoesh/63194/c6808453-1a32-458f-b2e7-2bf981c5690d.ini
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/food/filtered_openfood_sf/objects.bin to /var/tmp/graphlab-zoesh/63194/5cd8f0da-d738-4891-bfa2-93d544a59047.bin
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/food/filtered_openfood_sf/m_ec0c66b259e235f5.frame_idx to /var/tmp/graphlab-zoesh/63194/5e5af31f-256c-4bd9-a383-48915318546e.frame_idx
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/food/filtered_openfood_sf/m_ec0c66b259e235f5.sidx to /var/tmp/graphlab-zoesh/63194/df7c1f22-0a4e-4b9e-90a1-b3066400a3c5.sidx
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/food/filtered_openfood_sf/m_ec0c66b259e235f5.0000 to /var/tmp/graphlab-zoesh/63194/e8e03a00-cb2a-4548-a0d8-82318152e9dc.0000
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/food/filtered_openfood_sf/m

In [3]:
openfood_sf['product_features'] = openfood_sf['product_name'] + ', ' + openfood_sf['generic_name'] + ' ' + openfood_sf['categories'] + ' ' + openfood_sf['packaging'] + ' ' + openfood_sf['brands'] + ' ' + openfood_sf['labels'] + ' ' + openfood_sf['ingredients_text'] + ' ' + openfood_sf['allergens'] + ' ' + openfood_sf['additives']
openfood_sf['product_tags'] = openfood_sf['product_features']
tagger = graphlab.autotagger.create(openfood_sf, tag_name='product_tags')
openfood_similar_products = tagger.tag(openfood_sf, query_name='product_features', k=4)
openfood_similar_products = openfood_similar_products[openfood_similar_products['product_features'] != openfood_similar_products['product_tags']]
openfood_similar_products['product_name'] = openfood_similar_products['product_tags'].apply(lambda x: x.split(',')[0])
openfood_similar_products_groupby = openfood_similar_products.groupby(key_columns='product_features_id', operations={'similar_foods': graphlab.aggregate.CONCAT('product_name')})
openfood_sf = openfood_sf.add_row_number('row_id')
openfood_sf = openfood_sf.join(openfood_similar_products_groupby, how='left', on={'row_id':'product_features_id'})
openfood_sf = openfood_sf.fillna('similar_foods', ['','',''])

PROGRESS: Starting brute force nearest neighbors model training.
PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1            | 2169    | 0.0562746   | 11.177ms     |
PROGRESS: | Done         |         | 100         | 464.694ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [4]:
openfood_sf.select_columns(['product_name','similar_foods']).unpack('similar_foods')


product_name,similar_foods.0,similar_foods.1,similar_foods.2
Flute,Granary baguette,Arborio Risotto,Sweet and fragrant HomMali jasmine rice ...
Cauliflower,Cauliflower Florets,Sweetheart Cabbage,Courgettes
Spring Onions,Spring Greens,Trimmed Spring Onions,Swede
Luxury Christmas Pudding,Luxury 8 month maturedchristmas pudding ...,Luxury Fruit & NutChristmas Pudding ...,Cognac Laced ChristmasPudding ...
Trader Joe's sea saltpotato chips ...,Palak Paneer,Dark Chocolate Mints,Joe-Joes
,,,
Large flat mushrooms,Closed cup chestnutmushrooms ...,Closed Cup whiteMushrooms ...,Jumbo Mushrooms
,,,
British Beef BraisingSteak ...,Prime Irish Beef RibeyeSteak ...,2 Mediterranean SeabassFillets ...,2 Thick Cut Pork Chops
,,,


In [ ]:
visual_features = graphlab.SFrame()
visual_features['visual_features'] = graphlab.SArray('http://s3.amazonaws.com/dato-datasets/food/openfood_extracted_features')
visual_features = visual_features.add_row_number('row_number').filter_by(dedup_sf['row_number'], 'row_number')
openfood_sf['visual_features'] = visual_features['visual_features']
#pretrained_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#openfood_sf['visual_features'] = pretrained_model.extract_features(openfood_sf)
m = graphlab.nearest_neighbors.create(openfood_sf, features=['visual_features'])
similar_images = nearest_neighbor_model.query(images, k = 2)